In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/_7/rtr2wt550hl8ld3wr8pmy6gc0000gn/T/ipykernel_1402/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [8]:
import os
import pandas as pd  

# Define the file path
file_path = "../data/kg_train.csv"

# Check if the file exists before reading
if os.path.exists(file_path):
    data = pd.read_csv(file_path, encoding="latin-1")  
    data = data.head(1000)  # Reduce dataset for faster development
    print("File loaded successfully.")
else:
    print(f"Error: File not found at {file_path}. Please check the path.")



File loaded successfully.


In [9]:
data

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
...,...,...
995,So what's the latest? It sounds contradictory ...,0
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1
997,Barb I will call to explain. Are you back in t...,0
998,Yang on travelNot free tonite.May work tomorrow,0


### Let's divide the training and test set into two partitions

In [10]:
import os
import pandas as pd  
from sklearn.model_selection import train_test_split

# Split into training (80%) and test (20%) sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [11]:
train_data

,text,label
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1
535,I have not been able to reach oscar this am. W...,0
695,; Huma Abedin B6I'm checking with Pat on the 5...,0
557,I can have it announced here on Monday - can't...,0
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1
...,...,...
106,7653 2612ADAMA IBRAHIM________________________...,1
270,What does that mean for our schedules?,0
860,"Dear Friend,My Compliment to you,I guess this ...",1
435,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...,1


In [12]:
test_data

,text,label
521,Dear Sir=2C I wish you go through this offer t...,1
737,To take your mind off the Balkans for a second...,0
740,Pls keep the updates coming!,0
660,</STRONG><STRONG>CHRIST BETHEL HOSPITAL<BR>11 ...,1
411,sbwhoeopFriday February 5 2010 7:11 AMHRe: Bra...,0
...,...,...
408,Sorry yes exactlyWe have shy tomorrow at 10am ...,0
332,DEAR=2CGOOD DAY=2EI KNOW THIS MESSAGE WILL COM...,1
208,FYI,0
613,Greetings Dear Friend Please Permit me to cont...,1


## Data Preprocessing

In [13]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amanialshaikh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [16]:
import re
from html.parser import HTMLParser

def clean_html(html):
    # 1. Remove inline JavaScript/CSS
    html = re.sub(r'<(script|style).*?>.*?</\1>', '', html, flags=re.DOTALL)
    
    # 2. Remove HTML comments
    html = re.sub(r'<!--.*?-->', '', html, flags=re.DOTALL)
    
    # 3. Remove remaining HTML tags
    html = re.sub(r'<.*?>', '', html)

    # 4. Remove extra whitespaces
    html = re.sub(r'\s+', ' ', html).strip()
    
    return html

# Example usage
html_content = """
<html>
<head>
    <script>console.log('Hello');</script>
    <style>body {color: red;}</style>
</head>
<body>
    <!-- This is a comment -->
    <h1>Welcome</h1>
    <p>This is a <b>test</b>.</p>
</body>
</html>
"""

cleaned_text = clean_html(html_content)
print(cleaned_text)  # Output: "Welcome This is a test."



Welcome This is a test.


In [17]:
train_data['clean_text'] = train_data['text'].apply(clean_html)

In [18]:
test_data['clean_text'] = test_data['text'].apply(clean_html)

In [19]:
test_data

,text,label,clean_text
521,Dear Sir=2C I wish you go through this offer t...,1,Dear Sir=2C I wish you go through this offer t...
737,To take your mind off the Balkans for a second...,0,To take your mind off the Balkans for a second...
740,Pls keep the updates coming!,0,Pls keep the updates coming!
660,</STRONG><STRONG>CHRIST BETHEL HOSPITAL<BR>11 ...,1,"CHRIST BETHEL HOSPITAL11 RUE ABOBOTE,ABIDJANIV..."
411,sbwhoeopFriday February 5 2010 7:11 AMHRe: Bra...,0,sbwhoeopFriday February 5 2010 7:11 AMHRe: Bra...
...,...,...,...
408,Sorry yes exactlyWe have shy tomorrow at 10am ...,0,Sorry yes exactlyWe have shy tomorrow at 10am ...
332,DEAR=2CGOOD DAY=2EI KNOW THIS MESSAGE WILL COM...,1,DEAR=2CGOOD DAY=2EI KNOW THIS MESSAGE WILL COM...
208,FYI,0,FYI
613,Greetings Dear Friend Please Permit me to cont...,1,Greetings Dear Friend Please Permit me to cont...


In [20]:
train_data

,text,label,clean_text
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,"----------- REGARDS, MR NELSON SMITH.KINDLY RE..."
535,I have not been able to reach oscar this am. W...,0,I have not been able to reach oscar this am. W...
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,; Huma Abedin B6I'm checking with Pat on the 5...
557,I can have it announced here on Monday - can't...,0,I can have it announced here on Monday - can't...
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...
...,...,...,...
106,7653 2612ADAMA IBRAHIM________________________...,1,7653 2612ADAMA IBRAHIM________________________...
270,What does that mean for our schedules?,0,What does that mean for our schedules?
860,"Dear Friend,My Compliment to you,I guess this ...",1,"Dear Friend,My Compliment to you,I guess this ..."
435,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...,1,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

## Now let's work on removing stopwords
Remove the stopwords.

In [21]:
import re
import nltk
from nltk.corpus import stopwords
from html.parser import HTMLParser

# Download stopwords from NLTK
nltk.download('stopwords')

def remove_stopwords(html):
    stop_words = set(stopwords.words('english'))
    words = html.split()  # Tokenize the text
    filtered_words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    return " ".join(filtered_words)  # Return the cleaned text


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amanialshaikh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
train_data['clean_text_no_stopwords'] = train_data['clean_text'].apply(remove_stopwords)
test_data['clean_text_no_stopwords'] = test_data['clean_text'].apply(remove_stopwords)

In [23]:
train_data

,text,label,clean_text,clean_text_no_stopwords
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...","----------- REGARDS, MR NELSON SMITH.KINDLY RE..."
535,I have not been able to reach oscar this am. W...,0,I have not been able to reach oscar this am. W...,able reach oscar am. supposed send pdb 11. u r...
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,; Huma Abedin B6I'm checking with Pat on the 5...,; Huma Abedin B6I'm checking Pat 50k work Jack...
557,I can have it announced here on Monday - can't...,0,I can have it announced here on Monday - can't...,announced Monday - can't today
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...,BANK AFRICAAGENCE SAN PEDRO14 BP 1210 San Pedr...
...,...,...,...,...
106,7653 2612ADAMA IBRAHIM________________________...,1,7653 2612ADAMA IBRAHIM________________________...,7653 2612ADAMA IBRAHIM________________________...
270,What does that mean for our schedules?,0,What does that mean for our schedules?,mean schedules?
860,"Dear Friend,My Compliment to you,I guess this ...",1,"Dear Friend,My Compliment to you,I guess this ...","Dear Friend,My Compliment you,I guess letter m..."
435,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...,1,Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...,Dear PRESIDENT=2FDIRECTOR=2C name Mr=2E Michea...


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [24]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/amanialshaikh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/amanialshaikh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [25]:
from nltk.stem import WordNetLemmatizer

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split()  # Tokenize the text into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    return " ".join(lemmatized_words)  # Return the lemmatized text


In [26]:
train_data['final_text'] = train_data['clean_text_no_stopwords'].apply(lemmatize_text)
test_data['final_text'] = test_data['clean_text_no_stopwords'].apply(lemmatize_text)

In [27]:
train_data = train_data[['label', 'final_text']]
test_data = test_data[['label', 'final_text']]

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [28]:
from collections import Counter
data_ham  = train_data[train_data['label'] == 0].copy()
data_spam = train_data[train_data['label'] == 1].copy()
words_data_ham  = data_ham['final_text']
words_data_spam = data_spam['final_text']
list_ham_words = []
for sublist in words_data_ham:
    for item in sublist.split():
        list_ham_words.append(item)
list_spam_words = []
for sublist in words_data_spam:
    for item in sublist.split():
        list_spam_words.append(item)
c_ham  = Counter(list_ham_words)
c_spam = Counter(list_spam_words)
df_hamwords_top10  = pd.DataFrame(c_ham.most_common(10),  columns=['word', 'count'])
df_spamwords_top10 = pd.DataFrame(c_spam.most_common(20), columns=['word', 'count'])
df_spamwords_top10

,word,count
0,money,597
1,account,430
2,fund,409
3,bank,295
4,business,264
5,contact,228
6,transfer,220
7,country,213
8,transaction,213
9,next,209


## Extra features

## How would work the Bag of Words with Count Vectorizer concept?

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
# Assuming train_data is already created and contains 'final_text'
bow_transformer = CountVectorizer(min_df=0.005, max_df=0.7).fit(train_data['final_text'])

print('Number of words:', len(bow_transformer.vocabulary_))


Number of words: 2943



## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [30]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Sample corpus (you can replace this with your dataset)
documents = [
    "The quick brown fox jumped over the lazy dog",
    "The fox is quick and brown",
    "The dog barked loudly"
]

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Vectorize the dataset (transform the documents into TF-IDF features)
X_tfidf = tfidf_vectorizer.fit_transform(documents)

# Convert the result into a DataFrame for easy interpretation (optional)
tfidf_matrix = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Print the shape of the TF-IDF matrix
print("Shape of the TF-IDF matrix:", X_tfidf.shape)

# Optional: Print the first few rows of the matrix
print(tfidf_matrix.head())


Shape of the TF-IDF matrix: (3, 12)
       and    barked     brown       dog       fox       is    jumped  \
0  0.00000  0.000000  0.293622  0.293622  0.293622  0.00000  0.386077   
1  0.49483  0.000000  0.376331  0.000000  0.376331  0.49483  0.000000   
2  0.00000  0.584483  0.000000  0.444514  0.000000  0.00000  0.000000   

       lazy    loudly      over     quick       the  
0  0.386077  0.000000  0.386077  0.293622  0.456047  
1  0.000000  0.000000  0.000000  0.376331  0.292254  
2  0.000000  0.584483  0.000000  0.000000  0.345205  


## And the Train a Classifier?

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd

# Sample data (replace with your dataset)
data = [
    {"text": "The quick brown fox jumped over the lazy dog", "label": 0},  # 0: Non-Spam
    {"text": "The fox is quick and brown", "label": 1},                   # 1: Spam
    {"text": "The dog barked loudly", "label": 0},                         # 0: Non-Spam
    {"text": "You won a free prize, claim it now", "label": 1},            # 1: Spam
    {"text": "Please transfer funds to your account", "label": 1},        # 1: Spam
    {"text": "Your bank account has been credited", "label": 1},          # 1: Spam
    {"text": "Meeting tomorrow at 10am", "label": 0}                       # 0: Non-Spam
]

# Create DataFrame
df = pd.DataFrame(data)

# Step 1: Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])

# Step 2: Get the target labels (0 or 1)
y = df['label']

# Step 3: Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Initialize and train the Logistic Regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Step 5: Make predictions on the test set
y_pred = classifier.predict(X_test)

# Step 6: Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Step 7: Print results
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 50.00%
Confusion Matrix:
[[0 1]
 [0 1]]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).